Hello welcome to the notebook where I will be implementing the DeLed Process with Gurobi

We will use data sourced from BTS on time reporting May 2023

Our goal is to generalize our model to accept any kind of delay cost function

In [1]:
import pandas as pd
pd.set_option('display.max_rows', 75)
import numpy as np
import json
import os
import matplotlib.pyplot as plt
import re

import gurobipy as gp
from gurobipy import GRB

from params import *

In [2]:
# May 2023 On Time performance
ontime_df = pd.read_csv('T_ONTIME_MARKETING.csv')

In [3]:
ontime_df['DEP_DELAY'] = ontime_df['DEP_DELAY'].apply(lambda x: 0 if x < 0 else x)

Dropping rows with an NA departure time, as we cannot do preprocessing on it. Removes $3693$ out of $616630$ total rows. Now at $612937$ rows a reduction of $0.5\%$

In [4]:
# Making FL_DATE a date time and removing na flights with no departing time
ontime_df['FL_DATE'] = pd.to_datetime(ontime_df['FL_DATE']).dt.time
ontime_df = ontime_df.dropna(subset=['DEP_TIME'])

C:\Users\nball\AppData\Local\Temp\ipykernel_15700\2280475078.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  ontime_df['FL_DATE'] = pd.to_datetime(ontime_df['FL_DATE']).dt.time


In [5]:
ontime_df['DEP_TIME'] = ontime_df['DEP_TIME'].fillna(0).astype(int).astype(str).str.zfill(4)
ontime_df['DEP_TIME'] = np.where(ontime_df['DEP_TIME'] == '0000', '0001', ontime_df['DEP_TIME'])
ontime_df['DEP_TIME'] = pd.to_datetime(ontime_df['DEP_TIME'], format='%H%M', errors='coerce').dt.time

In [6]:
ontime_df

,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,OP_UNIQUE_CARRIER,OP_CARRIER_AIRLINE_ID,ORIGIN,DEST,DEP_TIME,DEP_DELAY,DEP_DEL15,ARR_TIME,ARR_DELAY,ARR_DEL15,CANCELLED,FLIGHTS
0,5,1,1,00:00:00,9E,20363,ABY,ATL,07:40:00,0.0,0.0,844.0,-15.0,0.0,0.0,1.0
1,5,1,1,00:00:00,9E,20363,ABY,ATL,16:55:00,0.0,0.0,1747.0,-22.0,0.0,0.0,1.0
2,5,1,1,00:00:00,9E,20363,AEX,ATL,09:57:00,0.0,0.0,1229.0,-17.0,0.0,0.0,1.0
3,5,1,1,00:00:00,9E,20363,AEX,ATL,17:15:00,0.0,0.0,1950.0,-20.0,0.0,0.0,1.0
4,5,1,1,00:00:00,9E,20363,AGS,ATL,09:56:00,0.0,0.0,1047.0,-18.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
616625,5,31,3,00:00:00,ZW,20046,RST,ORD,11:28:00,9.0,0.0,1231.0,-3.0,0.0,0.0,1.0
616626,5,31,3,00:00:00,ZW,20046,SBN,ORD,13:05:00,0.0,0.0,1323.0,10.0,0.0,0.0,1.0
616627,5,31,3,00:00:00,ZW,20046,SBN,ORD,18:12:00,63.0,1.0,1755.0,45.0,1.0,0.0,1.0
616628,5,31,3,00:00:00,ZW,20046,SCE,ORD,10:28:00,0.0,0.0,1112.0,-38.0,0.0,0.0,1.0


In [7]:
# setting round length for 1 day
round_num = 1
ontime_round_df = ontime_df[ontime_df['DAY_OF_MONTH'] == round_num]
airlines = ontime_df['OP_UNIQUE_CARRIER'].unique().tolist()
airlines

['9E',
 'AA',
 'AS',
 'B6',
 'C5',
 'DL',
 'F9',
 'G4',
 'G7',
 'HA',
 'MQ',
 'NK',
 'OH',
 'OO',
 'PT',
 'QX',
 'UA',
 'WN',
 'YV',
 'YX',
 'ZW']

In [8]:
# random seed to be used throughout the entire notebook for reproduceability
seed = 845232
np.random.seed(seed)
def generate_slope(df, mean):
    '''
    PARAMS:
    df: dataframe of interest
    mean: mean parameter of the exponential distribution in this case 
    '''
    df_len = df.shape[0]
    df['SLOPE_LOSS'] = np.random.exponential(scale=mean, size=df_len)

def generate_intercept(df):
    df_len = df.shape[0]
    df['INTERCEPT_LOSS'] = np.random.exponential(scale=1/5, size=df_len)

In [9]:
generate_slope(ontime_df, 5)
generate_intercept(ontime_df)
ontime_df

,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,OP_UNIQUE_CARRIER,OP_CARRIER_AIRLINE_ID,ORIGIN,DEST,DEP_TIME,DEP_DELAY,DEP_DEL15,ARR_TIME,ARR_DELAY,ARR_DEL15,CANCELLED,FLIGHTS,SLOPE_LOSS,INTERCEPT_LOSS
0,5,1,1,00:00:00,9E,20363,ABY,ATL,07:40:00,0.0,0.0,844.0,-15.0,0.0,0.0,1.0,5.204536,0.015120
1,5,1,1,00:00:00,9E,20363,ABY,ATL,16:55:00,0.0,0.0,1747.0,-22.0,0.0,0.0,1.0,22.012739,0.332047
2,5,1,1,00:00:00,9E,20363,AEX,ATL,09:57:00,0.0,0.0,1229.0,-17.0,0.0,0.0,1.0,7.766236,0.203794
3,5,1,1,00:00:00,9E,20363,AEX,ATL,17:15:00,0.0,0.0,1950.0,-20.0,0.0,0.0,1.0,10.876341,0.071507
4,5,1,1,00:00:00,9E,20363,AGS,ATL,09:56:00,0.0,0.0,1047.0,-18.0,0.0,0.0,1.0,3.152512,0.132284
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
616625,5,31,3,00:00:00,ZW,20046,RST,ORD,11:28:00,9.0,0.0,1231.0,-3.0,0.0,0.0,1.0,3.465022,0.060654
616626,5,31,3,00:00:00,ZW,20046,SBN,ORD,13:05:00,0.0,0.0,1323.0,10.0,0.0,0.0,1.0,1.423169,0.588882
616627,5,31,3,00:00:00,ZW,20046,SBN,ORD,18:12:00,63.0,1.0,1755.0,45.0,1.0,0.0,1.0,3.646840,0.409766
616628,5,31,3,00:00:00,ZW,20046,SCE,ORD,10:28:00,0.0,0.0,1112.0,-38.0,0.0,0.0,1.0,9.477512,0.009279


In [10]:
def integrate_linear_loss(slope, intercept, time):
    '''
    Here just integrate the trapezoid from 0 to the time
    '''
    start = intercept
    end = intercept + slope * time
    return ((start + end) * time / 2)

avg = ontime_df['DEP_DELAY'].mean()
ontime_df['FIRST_LOSS'] = ontime_df.apply(lambda row: integrate_linear_loss(row['SLOPE_LOSS'], row['INTERCEPT_LOSS'], time=avg), axis=1)

In [11]:
def generate_step_function(mean):
    """
    Generate a random step function based over a specified time and the size of a step.
    We will be assuming that a plane is delayed for at MOST 24 hours

    Assumptions made, the cost of a delay is being modeled as a step function with 4 periods
    hr 0-1 is the first period a slighly lower step
    hr 1-3 is the second period with a slighlty higher step
    hr 3-6 sees a massive increase
    hr 6-24 sees a small step up from hr 3-6
    """

    step1 = np.random.exponential(mean)
    step2 = step1 + np.random.exponential(mean)
    step3 = step2 + np.random.exponential(4*mean)
    step4 = step3 + np.random.exponential(mean)

    return [(step1, 60), (step2, 180), (step3, 360), (step4, 1440)]

ontime_df['STEP_LOSS_FUNCTION'] = ontime_df.apply(lambda row: generate_step_function(5), axis=1)

In [12]:
def integrate_step(f, time):
    '''
    Calculate the area under a step function up to a specified limit `x_limit`.

    Parameters:
    step_function (list of tuples): Each tuple is (height, x) where `height` the step height up to x and `x` represents the end of the step interval.
    x_limit (float): The upper limit of x up to which the area should be calculated.
    
    Returns:
    float: Area under the step function up to `x_limit`.
    '''
    if time > 24:
        raise ValueError("Value Too Large")
    total = 0
    previous = 0
    for height, x in f:
        if x > time:
            width = time - previous
            total += height * width
            break
        else:
            width = x - previous
            total += height * width
            previous = x
    return total
ontime_df['STEP_FIRST_LOSS'] = ontime_df.apply(lambda row: integrate_step(row['STEP_LOSS_FUNCTION'], avg), axis=1)
ontime_df

,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,OP_UNIQUE_CARRIER,OP_CARRIER_AIRLINE_ID,ORIGIN,DEST,DEP_TIME,DEP_DELAY,...,ARR_TIME,ARR_DELAY,ARR_DEL15,CANCELLED,FLIGHTS,SLOPE_LOSS,INTERCEPT_LOSS,FIRST_LOSS,STEP_LOSS_FUNCTION,STEP_FIRST_LOSS
0,5,1,1,00:00:00,9E,20363,ABY,ATL,07:40:00,0.0,...,844.0,-15.0,0.0,0.0,1.0,5.204536,0.015120,460.739141,"[(7.161746679428007, 60), (15.472704997565014,...",95.274278
1,5,1,1,00:00:00,9E,20363,ABY,ATL,16:55:00,0.0,...,1747.0,-22.0,0.0,0.0,1.0,22.012739,0.332047,1952.276296,"[(0.8167706887388332, 60), (4.061759458536781,...",10.865679
2,5,1,1,00:00:00,9E,20363,AEX,ATL,09:57:00,0.0,...,1229.0,-17.0,0.0,0.0,1.0,7.766236,0.203794,689.928270,"[(1.5554902869049894, 60), (2.0299975572980786...",20.693027
3,5,1,1,00:00:00,9E,20363,AEX,ATL,17:15:00,0.0,...,1950.0,-20.0,0.0,0.0,1.0,10.876341,0.071507,963.374812,"[(5.172197416508951, 60), (8.514002756695282, ...",68.806870
4,5,1,1,00:00:00,9E,20363,AGS,ATL,09:56:00,0.0,...,1047.0,-18.0,0.0,0.0,1.0,3.152512,0.132284,280.718646,"[(14.509871056768844, 60), (32.88118533165541,...",193.027980
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
616625,5,31,3,00:00:00,ZW,20046,RST,ORD,11:28:00,9.0,...,1231.0,-3.0,0.0,0.0,1.0,3.465022,0.060654,307.419090,"[(5.950903332223866, 60), (17.442796915296856,...",79.166165
616626,5,31,3,00:00:00,ZW,20046,SBN,ORD,13:05:00,0.0,...,1323.0,10.0,0.0,0.0,1.0,1.423169,0.588882,133.767120,"[(0.3363263665115674, 60), (2.5333960196630847...",4.474223
616627,5,31,3,00:00:00,ZW,20046,SBN,ORD,18:12:00,63.0,...,1755.0,45.0,1.0,0.0,1.0,3.646840,0.409766,328.152061,"[(0.1436249134200508, 60), (19.472430409081472...",1.910674
616628,5,31,3,00:00:00,ZW,20046,SCE,ORD,10:28:00,0.0,...,1112.0,-38.0,0.0,0.0,1.0,9.477512,0.009279,838.767695,"[(3.7464126874717536, 60), (13.81408141654114,...",49.839345


In [13]:
avg

13.303218122639432

A reminder of where I am at right now. Currently I have established a lost function for each flight, as well as a "first loss" parameter that was obtained by integrating the function up to the average delay time.

We will now use this "first loss" parameter to simulate flight importance

In [27]:
mean_delay_df = ontime_df.groupby('OP_UNIQUE_CARRIER').agg({
    'DEP_DELAY': ['mean', 'std', 'count'],
    'FIRST_LOSS': ['mean', 'std', 'count'],
    'STEP_FIRST_LOSS': ['mean', 'std', 'count']
}).reset_index()
mean_delay_df = mean_delay_df.reset_index()
mean_delay_df

index OP_UNIQUE_CARRIER  DEP_DELAY                     FIRST_LOSS  \
                                 mean        std   count        mean   
0      0                9E   5.383442  28.375402   15992  443.514950   
1      1                AA  19.520283  79.549809   79403  446.403807   
2      2                AS   7.892869  25.741324   20573  444.599519   
3      3                B6  20.796534  56.792017   24466  445.685173   
4      4                C5  15.752688  55.096079    5673  448.868041   
5      5                DL   9.887650  41.705124   84050  446.245475   
6      6                F9  26.422980  69.352758   13951  436.593486   
7      7                G4  15.676517  67.315908    9407  444.614421   
8      8                G7   8.745015  31.681041    3561  437.668433   
9      9                HA  22.672373  57.158683    6718  456.772841   
10    10                MQ   8.735921  33.112054   17597  449.671884   
11    11                NK  17.246531  45.108267   22338  447.214208   
12    12                OH   7.955315  38.945906   16001  445.864570   
13    13                OO   9.441044  45.603992   55999  446.831812   
14    14                PT   6.505635  38.135498    8518  449.867214   
15    15                QX   8.316430  31.250158    5837  441.360273   
16    16                UA  15.961119  49.213429   61856  442.799033   
17    17                WN  12.233346  26.508539  121909  445.423356   
18    18                YV  34.476002  85.438538    7334  439.466446   
19    19                YX   4.218048  24.307671   26540  445.258362   
20    20                ZW  16.198772  73.142800    5212  443.172798   

                       STEP_FIRST_LOSS                     
           std   count            mean        std   count  
0   433.290345   15992       65.531766  65.710448   15992  
1   445.528759   79403       66.977485  67.160197   79403  
2   438.357976   20573       66.627678  66.847512   20573  
3   438.934533   24466       66.564528  66.328839   24466  
4   444.301946    5673       67.032057  66.916782    5673  
5   443.006035   84050       66.531934  66.874393   84050  
6   431.941071   13951       66.726429  66.408511   13951  
7   438.726604    9407       65.369588  65.866451    9407  
8   427.498420    3561       64.657815  65.083581    3561  
9   450.074034    6718       66.343480  66.340487    6718  
10  448.585086   17597       67.166261  67.136242   17597  
11  439.506891   22338       66.248728  66.643953   22338  
12  444.034394   16001       66.789898  66.257454   16001  
13  441.059272   56001       66.575292  66.373184   56001  
14  446.563669    8518       66.452619  65.846139    8518  
15  436.966240    5837       67.159126  67.758727    5837  
16  441.946769   61856       66.584859  66.288162   61856  
17  443.573183  121909       66.375953  66.472298  121909  
18  436.773140    7334       65.758261  65.901466    7334  
19  437.539397   26540       66.773476  66.917080   26540  
20  435.648353    5212       65.631498  64.728957    5212

In [ ]:
mean_delay_df['DEP_DELAY', 'std_error'] = mean_delay_df['DEP_DELAY', 'std']/mean_delay_df['DEP_DELAY', 'count']
mean_delay_df['FIRST_LOSS', 'std_error'] = mean_delay_df['FIRST_LOSS', 'std']/mean_delay_df['FIRST_LOSS', 'count']
mean_delay_df['STEP_FIRST_LOSS', 'std_error'] = mean_delay_df['STEP_FIRST_LOSS', 'std']/mean_delay_df['STEP_FIRST_LOSS', 'count']
mean_delay_df

index OP_UNIQUE_CARRIER  DEP_DELAY                     FIRST_LOSS  \
                                 mean        std   count        mean   
0      0                9E   5.383442  28.375402   15992  443.514950   
1      1                AA  19.520283  79.549809   79403  446.403807   
2      2                AS   7.892869  25.741324   20573  444.599519   
3      3                B6  20.796534  56.792017   24466  445.685173   
4      4                C5  15.752688  55.096079    5673  448.868041   
5      5                DL   9.887650  41.705124   84050  446.245475   
6      6                F9  26.422980  69.352758   13951  436.593486   
7      7                G4  15.676517  67.315908    9407  444.614421   
8      8                G7   8.745015  31.681041    3561  437.668433   
9      9                HA  22.672373  57.158683    6718  456.772841   
10    10                MQ   8.735921  33.112054   17597  449.671884   
11    11                NK  17.246531  45.108267   22338  447.214208   
12    12                OH   7.955315  38.945906   16001  445.864570   
13    13                OO   9.441044  45.603992   55999  446.831812   
14    14                PT   6.505635  38.135498    8518  449.867214   
15    15                QX   8.316430  31.250158    5837  441.360273   
16    16                UA  15.961119  49.213429   61856  442.799033   
17    17                WN  12.233346  26.508539  121909  445.423356   
18    18                YV  34.476002  85.438538    7334  439.466446   
19    19                YX   4.218048  24.307671   26540  445.258362   
20    20                ZW  16.198772  73.142800    5212  443.172798   

                       STEP_FIRST_LOSS                    DEP_DELAY  \
           std   count            mean        std   count std_error   
0   433.290345   15992       65.531766  65.710448   15992  0.001774   
1   445.528759   79403       66.977485  67.160197   79403  0.001002   
2   438.357976   20573       66.627678  66.847512   20573  0.001251   
3   438.934533   24466       66.564528  66.328839   24466  0.002321   
4   444.301946    5673       67.032057  66.916782    5673  0.009712   
5   443.006035   84050       66.531934  66.874393   84050  0.000496   
6   431.941071   13951       66.726429  66.408511   13951  0.004971   
7   438.726604    9407       65.369588  65.866451    9407  0.007156   
8   427.498420    3561       64.657815  65.083581    3561  0.008897   
9   450.074034    6718       66.343480  66.340487    6718  0.008508   
10  448.585086   17597       67.166261  67.136242   17597  0.001882   
11  439.506891   22338       66.248728  66.643953   22338  0.002019   
12  444.034394   16001       66.789898  66.257454   16001  0.002434   
13  441.059272   56001       66.575292  66.373184   56001  0.000814   
14  446.563669    8518       66.452619  65.846139    8518  0.004477   
15  436.966240    5837       67.159126  67.758727    5837  0.005354   
16  441.946769   61856       66.584859  66.288162   61856  0.000796   
17  443.573183  121909       66.375953  66.472298  121909  0.000217   
18  436.773140    7334       65.758261  65.901466    7334  0.011650   
19  437.539397   26540       66.773476  66.917080   26540  0.000916   
20  435.648353    5212       65.631498  64.728957    5212  0.014034   

   FIRST_LOSS STEP_FIRST_LOSS  
    std_error       std_error  
0    0.027094        0.004109  
1    0.005611        0.000846  
2    0.021307        0.003249  
3    0.017941        0.002711  
4    0.078319        0.011796  
5    0.005271        0.000796  
6    0.030961        0.004760  
7    0.046638        0.007002  
8    0.120050        0.018277  
9    0.066995        0.009875  
10   0.025492        0.003815  
11   0.019675        0.002983  
12   0.027750        0.004141  
13   0.007876        0.001185  
14   0.052426        0.007730  
15   0.074861        0.011608  
16   0.007145        0.001072  
17   0.003639        0.000545  
18   0.059555        0.008986  
19   0.016486        0.002521  
20   0.0

This cell is a method I used to calculate importance, it is super duper slow, takes 40+ minutes to run on my computer. However, I figured out a fast way to do it, which is the actual data set we will be using.

In [ ]:
'''
def assign_linear_importance(row):
    airline = row['OP_UNIQUE_CARRIER']
    loss = row['FIRST_LOSS']
    
    # Access the mean and std_error for the specific airline in FIRST_LOSS
    upper_bound = (mean_delay_df.loc[mean_delay_df['OP_UNIQUE_CARRIER'] == airline, ('FIRST_LOSS', 'mean')].values[0] +
                   mean_delay_df.loc[mean_delay_df['OP_UNIQUE_CARRIER'] == airline, ('FIRST_LOSS', 'std')].values[0])/3
    lower_bound = (mean_delay_df.loc[mean_delay_df['OP_UNIQUE_CARRIER'] == airline, ('FIRST_LOSS', 'mean')].values[0] -
                   mean_delay_df.loc[mean_delay_df['OP_UNIQUE_CARRIER'] == airline, ('FIRST_LOSS', 'std')].values[0])/3

    if loss < lower_bound:
        return 'L'
    elif loss < upper_bound:
        return 'M'
    else:
        return 'H'

def assign_step_importance(row):
    airline = row['OP_UNIQUE_CARRIER']
    loss = row['STEP_FIRST_LOSS']
    
    # Access the mean and std_error for the specific airline in STEP_FIRST_LOSS
    upper_bound = (mean_delay_df.loc[mean_delay_df['OP_UNIQUE_CARRIER'] == airline, ('STEP_FIRST_LOSS', 'mean')].values[0] +
                   mean_delay_df.loc[mean_delay_df['OP_UNIQUE_CARRIER'] == airline, ('STEP_FIRST_LOSS', 'std_error')].values[0])
    lower_bound = (mean_delay_df.loc[mean_delay_df['OP_UNIQUE_CARRIER'] == airline, ('STEP_FIRST_LOSS', 'mean')].values[0] -
                   mean_delay_df.loc[mean_delay_df['OP_UNIQUE_CARRIER'] == airline, ('STEP_FIRST_LOSS', 'std_error')].values[0])

    if loss < lower_bound:
        return 'L'
    elif loss < upper_bound:
        return 'M'
    else:
        return 'H'

# Applying the functions
ontime_df['LINEAR_IMPORTANCE'] = ontime_df.apply(assign_linear_importance, axis=1)
ontime_df['STEP_IMPORTANCE'] = ontime_df.apply(assign_step_importance, axis=1)
'''

In [43]:
ontime_df

,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,OP_UNIQUE_CARRIER,OP_CARRIER_AIRLINE_ID,ORIGIN,DEST,DEP_TIME,DEP_DELAY,...,ARR_DEL15,CANCELLED,FLIGHTS,SLOPE_LOSS,INTERCEPT_LOSS,FIRST_LOSS,STEP_LOSS_FUNCTION,STEP_FIRST_LOSS,LINEAR_IMPORTANCE,STEP_IMPORTANCE
0,5,1,1,00:00:00,9E,20363,ABY,ATL,07:40:00,0.0,...,0.0,0.0,1.0,5.204536,0.015120,460.739141,"[(7.161746679428007, 60), (15.472704997565014,...",95.274278,H,H
1,5,1,1,00:00:00,9E,20363,ABY,ATL,16:55:00,0.0,...,0.0,0.0,1.0,22.012739,0.332047,1952.276296,"[(0.8167706887388332, 60), (4.061759458536781,...",10.865679,H,L
2,5,1,1,00:00:00,9E,20363,AEX,ATL,09:57:00,0.0,...,0.0,0.0,1.0,7.766236,0.203794,689.928270,"[(1.5554902869049894, 60), (2.0299975572980786...",20.693027,H,L
3,5,1,1,00:00:00,9E,20363,AEX,ATL,17:15:00,0.0,...,0.0,0.0,1.0,10.876341,0.071507,963.374812,"[(5.172197416508951, 60), (8.514002756695282, ...",68.806870,H,H
4,5,1,1,00:00:00,9E,20363,AGS,ATL,09:56:00,0.0,...,0.0,0.0,1.0,3.152512,0.132284,280.718646,"[(14.509871056768844, 60), (32.88118533165541,...",193.027980,M,H
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
616625,5,31,3,00:00:00,ZW,20046,RST,ORD,11:28:00,9.0,...,0.0,0.0,1.0,3.465022,0.060654,307.419090,"[(5.950903332223866, 60), (17.442796915296856,...",79.166165,H,H
616626,5,31,3,00:00:00,ZW,20046,SBN,ORD,13:05:00,0.0,...,0.0,0.0,1.0,1.423169,0.588882,133.767120,"[(0.3363263665115674, 60), (2.5333960196630847...",4.474223,M,L
616627,5,31,3,00:00:00,ZW,20046,SBN,ORD,18:12:00,63.0,...,1.0,0.0,1.0,3.646840,0.409766,328.152061,"[(0.1436249134200508, 60), (19.472430409081472...",1.910674,H,L
616628,5,31,3,00:00:00,ZW,20046,SCE,ORD,10:28:00,0.0,...,0.0,0.0,1.0,9.477512,0.009279,838.767695,"[(3.7464126874717536, 60), (13.81408141654114,...",49.839345,H,L


In [44]:
ontime_df['LINEAR_IMPORTANCE'].value_counts()

LINEAR_IMPORTANCE
H    316607
M    295882
L       448
Name: count, dtype: int64

In [45]:
ontime_df.to_csv('ontime_marketing_preprocessing.csv')

Expanding the dataset so that I can directly access the columns

In [48]:
expanded_df = pd.DataFrame({
    'Index': mean_delay_df['index'],
    'OP_UNIQUE_CARRIER': mean_delay_df['OP_UNIQUE_CARRIER'],

    # DEP_DELAY Statistics
    'DEP_DELAY_mean': mean_delay_df['DEP_DELAY']['mean'],
    'DEP_DELAY_std': mean_delay_df['DEP_DELAY']['std'],
    'DEP_DELAY_count': mean_delay_df['DEP_DELAY']['count'],
    'DEP_DELAY_std_error': mean_delay_df['DEP_DELAY']['std_error'],

    # FIRST_LOSS Statistics
    'FIRST_LOSS_mean': mean_delay_df['FIRST_LOSS']['mean'],
    'FIRST_LOSS_std': mean_delay_df['FIRST_LOSS']['std'],
    'FIRST_LOSS_count': mean_delay_df['FIRST_LOSS']['count'],
    'FIRST_LOSS_std_error': mean_delay_df['FIRST_LOSS']['std_error'],

    # STEP_FIRST_LOSS Statistics
    'STEP_FIRST_LOSS_mean': mean_delay_df['STEP_FIRST_LOSS']['mean'],
    'STEP_FIRST_LOSS_std': mean_delay_df['STEP_FIRST_LOSS']['std'],
    'STEP_FIRST_LOSS_count': mean_delay_df['STEP_FIRST_LOSS']['count'],
    'STEP_FIRST_LOSS_std_error': mean_delay_df['STEP_FIRST_LOSS']['std_error'],
})

# Display the expanded DataFrame
expanded_df.head()

,Index,OP_UNIQUE_CARRIER,DEP_DELAY_mean,DEP_DELAY_std,DEP_DELAY_count,DEP_DELAY_std_error,FIRST_LOSS_mean,FIRST_LOSS_std,FIRST_LOSS_count,FIRST_LOSS_std_error,STEP_FIRST_LOSS_mean,STEP_FIRST_LOSS_std,STEP_FIRST_LOSS_count,STEP_FIRST_LOSS_std_error
0,0,9E,5.383442,28.375402,15992,0.001774,443.514950,433.290345,15992,0.027094,65.531766,65.710448,15992,0.004109
1,1,AA,19.520283,79.549809,79403,0.001002,446.403807,445.528759,79403,0.005611,66.977485,67.160197,79403,0.000846
2,2,AS,7.892869,25.741324,20573,0.001251,444.599519,438.357976,20573,0.021307,66.627678,66.847512,20573,0.003249
3,3,B6,20.796534,56.792017,24466,0.002321,445.685173,438.934533,24466,0.017941,66.564528,66.328839,24466,0.002711
4,4,C5,15.752688,55.096079,5673,0.009712,448.868041,444.301946,5673,0.078319,67.032057,66.916782,5673,0.011796


Assigning importance of each flight
After running the cell we have the following distribution:
LINEAR_IMPORTANCE:
L: 337274
M: 90831
H: 184802

STEP_IMPORTANCE:
L: 337724
M: 90682
H: 184531

In [57]:
merged_df = ontime_df.merge(expanded_df, on='OP_UNIQUE_CARRIER', how='left')
conditions = [
    merged_df['FIRST_LOSS'] < (merged_df['FIRST_LOSS_mean'] - merged_df['FIRST_LOSS_std'] / 5),
    (merged_df['FIRST_LOSS'] >= (merged_df['FIRST_LOSS_mean'] - merged_df['FIRST_LOSS_std'] / 5)) &
    (merged_df['FIRST_LOSS'] <= (merged_df['FIRST_LOSS_mean'] + merged_df['FIRST_LOSS_std'] / 5)),
    merged_df['FIRST_LOSS'] > (merged_df['FIRST_LOSS_mean'] + merged_df['FIRST_LOSS_std'] / 5)
]


# Define the choices corresponding to each condition
choices = ['L', 'M', 'H']

# Apply the conditions to create the LINEAR_IMPORTANCE column
merged_df['LINEAR_IMPORTANCE'] = np.select(conditions, choices, default='M')

conditions = [
    merged_df['STEP_FIRST_LOSS'] < (merged_df['STEP_FIRST_LOSS_mean'] - merged_df['STEP_FIRST_LOSS_std'] / 5),
    (merged_df['STEP_FIRST_LOSS'] >= (merged_df['STEP_FIRST_LOSS_mean'] - merged_df['STEP_FIRST_LOSS_std'] / 5)) &
    (merged_df['STEP_FIRST_LOSS'] <= (merged_df['STEP_FIRST_LOSS_mean'] + merged_df['STEP_FIRST_LOSS_std'] / 5)),
    merged_df['STEP_FIRST_LOSS'] > (merged_df['STEP_FIRST_LOSS_mean'] + merged_df['STEP_FIRST_LOSS_std'] / 5)
]

merged_df['STEP_IMPORTANCE'] = np.select(conditions, choices, default='M')
merged_df['STEP_IMPORTANCE'].value_counts()

STEP_IMPORTANCE
L    337724
H    184531
M     90682
Name: count, dtype: int64